In [92]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [93]:
df = pd.read_csv("dummy_classifier.csv")

In [94]:
#Configurando options do pandas para visualização
pd.set_option('display.max_columns', 500)

In [91]:
#Vamos comecar olhando descritivas da base, vamos notar que se trata de um dado complexo, com pouco contexto
#Esse tipo de dado é complexo para uma pessoa observar e encontrar padrões, por isso modelamos a informação
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.556348,0.380406,-0.373548,0.419505,0.013038,-0.388650,0.001836,-0.299414,0.003877,-0.118151,0.000944,0.001801,-0.199166,-0.004606,0.003031,-0.210832,0.003085,0.084875,0.001796,-0.001446,0.501030
std,1.159880,1.437376,1.060570,0.934494,0.841589,1.030115,0.998353,1.172385,1.003891,0.772854,1.002021,1.225262,1.137065,1.002853,0.999316,1.418103,1.000717,1.299358,1.000113,1.001888,0.500001
min,-4.823502,-7.334198,-5.213052,-3.900791,-3.657974,-6.030567,-4.366919,-5.981440,-4.388613,-3.705001,-4.104707,-4.990457,-5.826450,-4.517917,-4.289951,-5.662096,-4.403541,-5.197122,-4.137466,-4.434840,0.000000
25%,-0.219648,-0.449100,-0.977617,-0.235516,-0.585946,-0.920927,-0.670593,-1.084097,-0.674748,-0.612307,-0.673395,-0.876487,-0.950849,-0.683288,-0.668872,-1.210900,-0.670548,-0.758581,-0.679394,-0.676436,0.000000
50%,0.444283,0.494584,-0.554817,0.392519,0.027087,-0.277806,0.000491,-0.206833,0.004592,-0.113410,0.000003,-0.070469,-0.118963,-0.005968,0.006223,-0.201436,0.003973,0.117729,0.003835,0.000451,1.000000
75%,1.325171,1.329646,0.294473,0.958363,0.620886,0.284620,0.672774,0.587881,0.684367,0.356457,0.674134,0.847919,0.594264,0.675851,0.680672,0.807538,0.679180,0.938114,0.677689,0.674756,1.000000
max,5.397826,5.820011,4.882576,5.198659,3.340537,3.977398,4.292186,4.448398,4.367829,2.955692,4.427592,5.535576,3.959930,4.111248,4.178289,5.197292,4.359005,5.574640,4.151129,4.298875,1.000000


In [95]:
df["20"].value_counts()

1.0    50103
0.0    49897
Name: 20, dtype: int64

In [97]:
df.columns[-1]

'20'

In [98]:
target_name = df.columns[-1]
print(target_name)

20


In [45]:
#Baseado na descrição acima, percebemos que a coluna 20 é a unica que contém valores binários (0 ou 1)
#Sendo assim ela provavelmente representa nossa target - ou variável resposta
df[f"{target_name}"].value_counts()

1.0    50103
0.0    49897
Name: 20, dtype: int64

### Separando dos dados em conjuntos de treino e teste
Vamos dividir nossos dados em dois conjuntos: Treino e Teste.
Esse processo é importante para validarmos o processo de modelagem: 
- Treinamos utilizando o conjunto de <b>Treino</b>
- Verificamos as metricas de interesse no conjunto de <b>Teste</b>
- Podemos verificar as metricas também no conjunto de Treino com o objetivo de diagnosticar <i>overfitting</i>

Para essa separação, podemos utilizar regras simples, como 80% dos dados para treino e 20% para teste.

In [99]:
X_train, X_test, y_train, y_test = train_test_split(df.drop([f"{target_name}"],axis=1), #Removemos a target do conjunto de Treino
                                                   df[f"{target_name}"],# Target
                                                   test_size = 0.2 #Costumamos usar regras simples, como 80% treino e 20% teste
                                                   )

In [100]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(80000, 20) (80000,) (20000, 20) (20000,)


### Treinamento dos modelos
Utilizaremos o pacote [sklearn](https://scikit-learn.org/stable/) para realizar o treinamento e cálculo das métricas de avaliação.

Utilizar esse pacote nos permite seguir uma padronização, que torna a sintaxe bem simples:
- .fit(X,y) realiza o treino do modelo
- .predict(X) faz a predição do modelo. Em modelos de classificação, a saída será discreta (ex: 0 ou 1)
- .predict_proba(X) faz a predição do modelo de forma contínua, ou seja, a probabilidade de pertencer a cada uma das classes.


In [101]:
#Modelo simples utilizando Regressão Logistica
lr = LogisticRegression()
lr.fit(X_train,y_train) #Treina o modelo

train_performance = lr.predict(X_train)
test_performance = lr.predict(X_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [103]:
test_performance

array([1., 0., 0., ..., 1., 1., 1.])

In [104]:
lr

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

### Utilizando o classification_report para medir performance
O método classification_report(y_verdadeiro, y_predito) recebe a predição do modelo (discreta) e os valores reais referentes a essa predição. Com esses valores, conseguimos calcular:
- Precision (Precisão): Classificados positivo (1) e realmente positivos, dividido pela quantidade de positivos previstos (o quão bem eu acerto quem eu classifico como positivo?)
- Recall (Revocação): Classificados positivo (1) e realmente positivos, dividido pelo total de positivos (Quantos positivos eu acerto no total?)
- f1-score: Forma de agregar precisão e recall.

Quando vamos calcular essas métricas, devemos definir o que é o positivo para nós. No caso de classificação, costumamos falar que o positivo é quem realiza nosso evento, ou seja, marcados como 1. Utilizando essa perspectiva, devemos observar a linha referente a classe 1 na nossa classification report.

Nota: Nada impede de mudarmos o referencial e chamar o positivo de 0, os calculos poderão ser feitos da mesma forma, porém a interpretação será diferente.

In [105]:
print(classification_report(y_train, train_performance)) #Medimos a performance no treino para bsucar overfitting.

              precision    recall  f1-score   support

         0.0       0.72      0.73      0.73     40007
         1.0       0.73      0.72      0.72     39993

    accuracy                           0.73     80000
   macro avg       0.73      0.73      0.73     80000
weighted avg       0.73      0.73      0.73     80000



In [51]:
#Medimos a performance no teste. Essa é a que vale, pois são dados desconhecidos pelo modelo
print(classification_report(y_test, test_performance)) 

              precision    recall  f1-score   support

         0.0       0.72      0.73      0.73      9991
         1.0       0.73      0.72      0.72     10009

    accuracy                           0.72     20000
   macro avg       0.72      0.72      0.72     20000
weighted avg       0.72      0.72      0.72     20000



### Modelo em árvore, note o padrão sklearn torna a sintaxe semelhante

In [52]:
tree = DecisionTreeClassifier()
tree.fit(X_train,y_train)

train_performance = tree.predict(X_train)
test_performance = tree.predict(X_test)

In [53]:
tree

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [54]:
print(classification_report(y_train, train_performance)) #Sinal de overfitting!!

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     39906
         1.0       1.00      1.00      1.00     40094

    accuracy                           1.00     80000
   macro avg       1.00      1.00      1.00     80000
weighted avg       1.00      1.00      1.00     80000



In [55]:
print(classification_report(y_test, test_performance)) #Sinal de overfitting!!

              precision    recall  f1-score   support

         0.0       0.68      0.68      0.68      9991
         1.0       0.68      0.69      0.69     10009

    accuracy                           0.68     20000
   macro avg       0.68      0.68      0.68     20000
weighted avg       0.68      0.68      0.68     20000



### Ensemble

In [56]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

train_performance = rf.predict(X_train)
test_performance = rf.predict(X_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [57]:
rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [58]:
print(classification_report(y_train, train_performance)) #Sinal de Overfitting

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98     39906
         1.0       0.99      0.98      0.98     40094

    accuracy                           0.98     80000
   macro avg       0.98      0.98      0.98     80000
weighted avg       0.98      0.98      0.98     80000



In [59]:
print(classification_report(y_test, test_performance)) #Sinal de Overfitting

              precision    recall  f1-score   support

         0.0       0.75      0.80      0.77      9991
         1.0       0.78      0.73      0.76     10009

    accuracy                           0.77     20000
   macro avg       0.77      0.77      0.77     20000
weighted avg       0.77      0.77      0.77     20000



### Lidando com Overfitting - Ajuste de Hyperparametros
Uma forma comum de lidar com <i>overfitting</i> é ajustando os hyperparametros de forma que controlemos o quanto o modelo consegue se ajustar em nossos dados.

O <i>overfitting</i> normalmente é causado por uma complexidade do modelo, em modelos de árvore isso quer dizer:
- Muitos estimadores (muitas árvores): Em modelos de ensemble, adicionar muitos modelos pode não agregar na previsão resultante. Muitos modelos identicos podem gerar um viés e causar <i>overfitting</i>
- Árvores muito profundas: Cada nível de profundidade da árvore cria uma regra nova para separar nossos dados, ou seja, cada folha possui menos amostras a cada nível que crescemos a árvore. Sendo assim, em treino, regras que afetam poucas amostras são muito especificas, portanto podem estar muito vínculadas ao conjunto de treino e não generalizar.

Uma forma comum de fazer a busca dos melhores hyperparametros é através do Grid Search (Busca em grade).
- Montamos uma grade contendo os hyperparametros de interesse e os respectivos valores.
- O algoritmo vai separar a nossa base em "mini conjuntos" de treino e teste para testar cada combinação da grade
- A melhor combinação (a que maximizar o acerto) será a escolhida.

Exemplo: Vamos testar uma RandomForest com 10 e 20 árvores, onde cada árvore pode ter profundidade máxima de 3 ou 4. Sendo assim, temos:

|iteração|árvores|profundidade|
| ------------- |:-------------:| -----:|
|1|10|3|
|2|20|3|
|3|10|4|
|4|20|4|

Supondo que vamos separar nossos dados em 3 subconjuntos A, B e C para testar cada combinação:
Na iteração 1 teremos

|Conjunto de treino|Conjunto de Teste|Acerto (%)|
| ------------- |:-------------:| -----:|
|A+B|C|80|
|A+C|B|90|
|B+C|A|95|

Sendo assim, essa estratégia da iteração 1 possui um acerto médio de 88.3%. Repetimos o processo até encontrar a melhor combinação.

In [106]:
X_train.shape

(80000, 20)

In [107]:
dict_rf = {
            "n_estimators":[5,10,20], #Regular o número de estimadores
            "max_depth":[2,3,4] #Regular a profundidade das árvores
          }
gs = GridSearchCV(rf, #Objeto do classificador sklearn
                  param_grid = dict_rf, #Dicionario de hyperparametros
                  cv = 3, #Validação cruzada - vezes que o algoritmo vai separar o nosso dado e testar
                  verbose = 2 #Apenas para vermos o que está acontecendo (print de textos na tela)
                 )

gs.fit(X_train,y_train) #Fit do objeto GS, uma RF será treinada com cada uma das combinações de hyperparametros do dicionario para cada valor do CV

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] max_depth=2, n_estimators=5 .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...................... max_depth=2, n_estimators=5, total=   0.2s
[CV] max_depth=2, n_estimators=5 .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ...................... max_depth=2, n_estimators=5, total=   0.2s
[CV] max_depth=2, n_estimators=5 .....................................
[CV] ...................... max_depth=2, n_estimators=5, total=   0.2s
[CV] max_depth=2, n_estimators=10 ....................................
[CV] ..................... max_depth=2, n_estimators=10, total=   0.4s
[CV] max_depth=2, n_estimators=10 ....................................
[CV] ..................... max_depth=2, n_estimators=10, total=   0.5s
[CV] max_depth=2, n_estimators=10 ....................................
[CV] ..................... max_depth=2, n_estimators=10, total=   0.4s
[CV] max_depth=2, n_estimators=20 ....................................
[CV] ..................... max_depth=2, n_estimators=20, total=   0.9s
[CV] max_depth=2, n_estimators=20 ....................................
[CV] ..................... max_depth=2, n_estimators=20, total=   0.8s
[CV] max_depth=2, n_estimators=20 ....................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:   18.6s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='wa

In [83]:
rf_gs = gs.best_estimator_

In [84]:
rf_gs

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [85]:
test_performance = rf_gs.predict(X_test)
train_performance = rf_gs.predict(X_train)

In [86]:
print(classification_report(y_train, train_performance))

              precision    recall  f1-score   support

         0.0       0.75      0.78      0.77     39906
         1.0       0.78      0.74      0.76     40094

    accuracy                           0.76     80000
   macro avg       0.76      0.76      0.76     80000
weighted avg       0.76      0.76      0.76     80000



In [87]:
print(classification_report(y_test, test_performance)) 

              precision    recall  f1-score   support

         0.0       0.75      0.78      0.76      9991
         1.0       0.77      0.74      0.75     10009

    accuracy                           0.76     20000
   macro avg       0.76      0.76      0.76     20000
weighted avg       0.76      0.76      0.76     20000

